In [8]:
import time
import os


In [7]:
#Size of the file in bytes
os.path.getsize('chess_games.csv')

4378895157

In [13]:
from dask import dataframe as dd
start = time.time()
dask_df = dd.read_csv("chess_games.csv")
end = time.time()
print("Reading csv with dask takes: ",(end-start),"sec")

Reading csv with dask takes:  0.040029287338256836 sec


In [8]:

import pandas as pd
start = time.time()
df = pd.read_csv("chess_games.csv")
end = time.time()
print("Read csv with pandas: ",(end-start),"sec")

Read csv with pandas:  579.4639246463776 sec


In [ ]:
#Importing with pandas,Modin and Ray took too long

In [ ]:
#importing with Dask worked the best

In [14]:
from dask import dataframe as dd
df = dd.read_csv('chess_games.csv',delimiter=',')

In [15]:
df.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 15 entries, Event to AN
dtypes: object(11), float64(2), int64(2)

In [16]:
#No. of Rows
len(df.index)

6256184

In [17]:
#No, of Columns
len(df.columns)

15

In [18]:
# remove special character
df.columns=df.columns.str.replace('[#,@,&]','')

<ipython-input-18-554f595a0ef5>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df.columns=df.columns.str.replace('[#,@,&]','')


In [19]:

#To remove white space from columns
df.columns = df.columns.str.replace(' ', '')

In [20]:
data=df.columns
data

Index(['Event', 'White', 'Black', 'Result', 'UTCDate', 'UTCTime', 'WhiteElo',
       'BlackElo', 'WhiteRatingDiff', 'BlackRatingDiff', 'ECO', 'Opening',
       'TimeControl', 'Termination', 'AN'],
      dtype='object')

In [21]:
%%writefile testutility.py
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re


################
# File Reading #
################

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)


def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string) 
    return string

def col_header_val(df,table_config):
   
    
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

Overwriting testutility.py


In [22]:
%%writefile file.yaml 
file_type: csv
dataset_name: chess_games
file_name: chess_games
table_name: chess_games
inbound_delimiter: ","
outbound_delimiter: ","
skip_leading_rows: 1
columns:
    - Event
    - White
    - Black
    - Result
    - UTCDate
    - UTCTime
    - WhiteElo
    - BlackElo
    - WhiteRatingDiff
    - BlackRatingDiff
    - Opening
    - TimeControl
    - Termination
    - AN

Overwriting file.yaml


In [23]:
# Read config file
import testutility as util
config_data = util.read_config_file("file.yaml")

In [24]:
#inspecting data of config file
config_data

{'file_type': 'csv',
 'dataset_name': 'chess_games',
 'file_name': 'chess_games',
 'table_name': 'chess_games',
 'inbound_delimiter': ',',
 'outbound_delimiter': ',',
 'skip_leading_rows': 1,
 'columns': ['Event',
  'White',
  'Black',
  'Result',
  'UTCDate',
  'UTCTime',
  'WhiteElo',
  'BlackElo',
  'WhiteRatingDiff',
  'BlackRatingDiff',
  'Opening',
  'TimeControl',
  'Termination',
  'AN']}

In [25]:
import pandas as pd

In [26]:

file_type= config_data['file_type']
source_file= "./" + config_data['file_name'] + f'.{file_type}'

df= pd.read_csv(source_file, config_data['inbound_delimiter'])


In [27]:
util.col_header_val(df, config_data)

column name and column length validation failed
Following File columns are not in the YAML file ['eco']
Following YAML columns are not in the file uploaded []


0

In [28]:
print("columns of files are:" ,df.columns)
print("columns of YAML are:" ,config_data['columns'])

columns of files are: Index(['event', 'white', 'black', 'result', 'utcdate', 'utctime', 'whiteelo',
       'blackelo', 'whiteratingdiff', 'blackratingdiff', 'eco', 'opening',
       'timecontrol', 'termination', 'an'],
      dtype='object')
columns of YAML are: ['Event', 'White', 'Black', 'Result', 'UTCDate', 'UTCTime', 'WhiteElo', 'BlackElo', 'WhiteRatingDiff', 'BlackRatingDiff', 'Opening', 'TimeControl', 'Termination', 'AN']


In [30]:
if util.col_header_val(df, config_data)==0:
    print("validation is failed")
    
else:
    print("validation is passed")

column name and column length validation failed
Following File columns are not in the YAML file ['eco']
Following YAML columns are not in the file uploaded []
validation is failed


In [31]:
import datetime
import csv
import gzip

from dask import dataframe as dd
df = dd.read_csv("chess_games.csv",delimiter=',')

# Write csv in gz format in pipe separated text file (|)
df.to_csv('chess_games.csv.gz',
          sep='|',
          header=True,
          index=False,
          quoting=csv.QUOTE_ALL,
          compression='gzip',
          quotechar='"',
          doublequote=True,
          line_terminator='\n')

['C:/Users/Payal/week6/chess_games.csv.gz\\00.part',
 'C:/Users/Payal/week6/chess_games.csv.gz\\01.part',
 'C:/Users/Payal/week6/chess_games.csv.gz\\02.part',
 'C:/Users/Payal/week6/chess_games.csv.gz\\03.part',
 'C:/Users/Payal/week6/chess_games.csv.gz\\04.part',
 'C:/Users/Payal/week6/chess_games.csv.gz\\05.part',
 'C:/Users/Payal/week6/chess_games.csv.gz\\06.part',
 'C:/Users/Payal/week6/chess_games.csv.gz\\07.part',
 'C:/Users/Payal/week6/chess_games.csv.gz\\08.part',
 'C:/Users/Payal/week6/chess_games.csv.gz\\09.part',
 'C:/Users/Payal/week6/chess_games.csv.gz\\10.part',
 'C:/Users/Payal/week6/chess_games.csv.gz\\11.part',
 'C:/Users/Payal/week6/chess_games.csv.gz\\12.part',
 'C:/Users/Payal/week6/chess_games.csv.gz\\13.part',
 'C:/Users/Payal/week6/chess_games.csv.gz\\14.part',
 'C:/Users/Payal/week6/chess_games.csv.gz\\15.part',
 'C:/Users/Payal/week6/chess_games.csv.gz\\16.part',
 'C:/Users/Payal/week6/chess_games.csv.gz\\17.part',
 'C:/Users/Payal/week6/chess_games.csv.gz\\18.

In [32]:
#size of the gz format folder in bytes
os.path.getsize('chess_games.csv.gz')

28672

In [33]:
df.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 15 entries, Event to AN
dtypes: object(11), float64(2), int64(2)

In [34]:
data=df.columns
data

Index(['Event', 'White', 'Black', 'Result', 'UTCDate', 'UTCTime', 'WhiteElo',
       'BlackElo', 'WhiteRatingDiff', 'BlackRatingDiff', 'ECO', 'Opening',
       'TimeControl', 'Termination', 'AN'],
      dtype='object')

In [35]:
#No, of Columns
len(df.columns)

15

In [36]:
#No. of Rows
len(df.index)

6256184